Note: if you having trouble displaying the maps try running jupyter nbextension enable --py gmaps in your environment and retry.


Create a heat map that displays the humidity for every city from the part I of the homework.

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.




Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.


Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.



In [1]:
# Dependencies
import requests
import json
import gmaps
import pandas as pd

# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

Humidity Heatmaps for all the cities (from WeatherPy.ipynb Part I)

In [2]:
###Get dataframe to show City Name (unduplicated): weather1_data ###
#import dataframe weather1_data from 1st part of assignment (WeatherPy.ipynb)
%store -r weather1_data
#drop duplicate cities (same data)
weather1_data=weather1_data.drop_duplicates(subset=["City Name"])
weather1_data

,City Name,Latitude,Longitude,Temperature,Latitude Humidity,Cloudiness,Wind Speed
1,hithadhoo,-20.841587,85.603908,82.76,68.0,77.0,3.94
2,sao filipe,13.293488,-34.982080,75.88,60.0,0.0,6.08
4,bluff,-71.214122,172.363302,57.00,84.0,64.0,18.01
5,saint-philippe,-48.409808,62.306578,75.02,65.0,47.0,4.61
6,norman wells,68.737352,-120.152631,-13.00,77.0,90.0,3.51
...,...,...,...,...,...,...,...
486,gravdal,70.454049,13.348788,24.80,74.0,75.0,5.75
488,souillac,-51.565186,73.401185,82.40,74.0,40.0,6.91
495,ous,61.793925,61.297353,-17.07,88.0,100.0,6.85
496,qasigiannguit,66.282569,-48.212406,19.40,42.0,75.0,9.22


In [28]:
# Plot Heatmap
fig = gmaps.figure()
fig

# Store latitude and longitude in locations
locations = weather1_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
Humidity = weather1_data["Latitude Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100, point_radius=2)
heat_layer

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

Heatmap for first hotel found within each city (with ideal conditions for travel destination) located within 5000 meters of your coordinates.

In [44]:
###Cities selected based on ideal travel destination: ideal_df
#Narrow down ideal conditions of city 

#Ideal temperature Range (in Farenheit)
ideal_min_temp=70 
ideal_max_temp=80
#Ideal wind speed: speed less than... (in mph)
ideal_max_speed=10
#Ideal cloudiness: cloudiness less than... (%)
ideal_cloudiness=25

ideal_df=weather1_data[(weather1_data["Temperature"]>=ideal_min_temp) & (weather1_data["Temperature"]<=ideal_max_temp) & (weather1_data["Wind Speed"]<=ideal_max_speed) & (weather1_data["Cloudiness"]<=ideal_cloudiness)]
ideal_df

#Dataframe sorted ascending by least windy city.
idealbylowwind_df=ideal_df.sort_values(by="Wind Speed", ascending=True)
idealbylowwind_df

ideal_df.shape

(11, 7)

In [62]:
##Nearest hotel nearest specified co-ordinates
#Print: Hotel Name, Address, City Name, Country.

#Create new dataframe containing the City, Latitude, Longitude, Hotel Name,and Country.
nearby_hotel=ideal_df[["City Name","Latitude","Longitude"]]
nearby_hotel["Hotel Name"]=""
nearby_hotel["Address"]=""
nearby_hotel["Country"]=""
nearby_hotel

# Assign variable values for parameters
target_search="hotels"
radius=10000

#Set parameters (unchanged for each row)
params = {
    "keyword": target_search,
    "radius": radius,
    "key": g_key
}

# Find a hotel near the specified city (co-ordinates available already)
for index, row in nearby_hotel.iterrows():
    #Use latitude and longitude from df
    lat= row["Latitude"]
    lng= row["Longitude"]
    #params dictionary to update each iteration
    #change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    #Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and print url
    name_address = requests.get(base_url, params=params)
    
    # # #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    # # print(name_address.url)

    #convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    # try:
    #     nearby_hotel.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    # except (KeyError, IndexError):
    #     print("Name error")
    try:
        nearby_hotel.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Hotel Address error")
    # try:
    #     nearby_hotel.loc[index, "Country"] = name_address["results"][0]["rating"]
    # except (KeyError, IndexError):
    #     print("rating error??")


Hotel Address error
Hotel Address error
Hotel Address error
Hotel Address error
Hotel Address error
Hotel Address error
Hotel Address error
Hotel Address error
Hotel Address error


In [ ]:
#####ADD MARKERS TO GOOGLE MAP##### EXAMPLE

# # Create a list containing coordinates
# coordinates = [
#     (40.71, -74.00),
#     (30.26, -97.74),
#     (46.87, -96.78),
#     (47.60, -122.33),
#     (32.71, -117.16)
# ]

# # Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)

# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)
# fig